## Problem 1-4 : NorthWinddatset https://relational.fit.cvut.cz/dataset/Northwind

How to download the dataset

To download the dataset you need to create connection in your mysql workbench

The datasets are publicly available directly from MariaDB database.

* Open your favourite MariaDB client (MySQL Workbench)
Use following credentials:
```
    hostname: relational.fit.cvut.cz
    port: 3306
    username: guest
    password: relational
```
Drive link: https://drive.google.com/drive/folders/1FMi6g1KDHbn17ikJSi7bE9-THywlxhdu?usp=sharing

###`Q-1`: Rank Employee in terms of revenue generation. Show employee id, first name, revenue, and rank


###`Q-2`: Show All products cumulative sum of units sold each month.


###`Q-3`: Show Percentage of total revenue by each suppliers



###`Q-4:` Show Percentage of total orders by each suppliers



### `Q-5`:Show All Products Year Wise report of totalQuantity sold, percentage change from last year.





In [ ]:
  select e.EmployeeID, e.FirstName , sum(od.UnitPrice * od.Quantity) as revenue,
  rank() over( order by revenue desc) as EmpRank
  from nw_orders o join nw_order_details od on od.OrderID = o.OrderID
  join nw_employees e on e.EmployeeID = o.EmployeeID
  group by e.EmployeeID, e.FirstName
  order by EmpRank


In [ ]:
select monthname(OrderDate) as month_name,year(OrderDate) as year,sum(od.Quantity),
sum(sum(od.Quantity)) over(order by year,month_name ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW )
from nw_orders o join nw_order_details od on od.OrderID = o.OrderID
join nw_employees e on e.EmployeeID = o.EmployeeID
group by year,month_name

In [ ]:
select s.SupplierId,s.CompanyName,sum(od.UnitPrice*od.Quantity) as Revenue, sum(od.UnitPrice*od.Quantity) /
sum(sum(od.UnitPrice*od.Quantity)) over() * 100 as PercentTotalRevenue
from nw_suppliers s join nw_products p on p.SupplierId=s.SupplierID
join nw_order_details od on p.ProductID=od.ProductID
group by s.SupplierId, s.CompanyName
order by PercentTotalRevenue desc

In [ ]:
select s.SupplierId,s.CompanyName,sum(distinct od.OrderID) as total, sum(distinct od.OrderID) /
sum(sum(distinct od.OrderID)) over() * 100 as PercentTotalorder
from nw_suppliers s join nw_products p on p.SupplierId=s.SupplierID
join nw_order_details od on p.ProductID=od.ProductID
group by s.SupplierId, s.CompanyName
order by PercentTotalorder desc

In [ ]:
select *,100 *(total - lag(total) over(partition  by ProductId order by ProductId, Year))/lag(total)
over(partition  by ProductId order by ProductId, Year) as PercentageChange
from(select year(OrderDate) as year, od.ProductID,sum(od.Quantity) as total
from nw_orders o join nw_order_details od on od.OrderID = o.OrderID
join nw_employees e on e.EmployeeID = o.EmployeeID
group by year, od.ProductID
order by od.ProductID,year) as t

## Problem 6 - 15

For the next problems, you can get the dataset as well as the details [here](https://www.kaggle.com/datasets/thedevastator/drug-performance-evaluation?select=Drug_clean.csv) (Use the *cleaned* csv file).

### **`Problem-6:`** For each condition, what is the average satisfaction level of drugs that are "On Label" vs "Off Label"?


### **`Problem-7:`** For each drug type (RX, OTC, RX/OTC), what is the average ease of use and satisfaction level of drugs with a price above the median for their type?


### **`Problem 8:`** What is the cumulative distribution of EaseOfUse ratings for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by drug type and cumulative distribution. (Use the built-in method and the manual method by calculating on your own. For the manual method, use the "ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW" and see if you get the same results as the built-in method.)


### **`Problem 9:`** What is the median satisfaction level for each medical condition? Show the results in descending order by median satisfaction level. (Don't repeat the same rows of your result.)



### **`Problem 10:`** What is the running average of the price of drugs for each medical condition? Show the results in ascending order by medical condition and drug name.



### **`Problem 11:`** What is the percentage change in the number of reviews for each drug between the previous row and the current row? Show the results in descending order by percentage change.



### **`Problem 12:`** What is the percentage of total satisfaction level for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by drug type and percentage of total satisfaction.



### **`Problem 13:`** What is the cumulative sum of effective ratings for each medical condition and drug form combination? Show the results in ascending order by medical condition, drug form and the name of the drug.



### **`Problem-14:`** What is the rank of the average ease of use for each drug type (RX, OTC, RX/OTC)? Show the results in descending order by rank and drug type.



### **`Problem-15:`** For each condition, what is the average effectiveness of the top 3 most reviewed drugs?

In [ ]:
select * from (SELECT t.Condition,Indication,Satisfaction,
              round(avg(Satisfaction) over(partition by t.Condition,Indication ),2) as avg_satis,
rank() over(partition by t.Condition,Indication order by Satisfaction) as rnk
FROM phpmyadmin.drug as t) as n
where n.rnk=1

In [ ]:
select * from  (SELECT Type,
        AVG(EaseOfUse) OVER(PARTITION BY Type) AS avg_ease_of_use,
        AVG(Satisfaction) OVER(PARTITION BY Type) AS avg_satisfaction
 from (select *, PERCENTILE_CONT(0.5) WITHIN GROUP (
                ORDER BY Price
            ) OVER (PARTITION BY Type) AS median_price from drug
where Type in ('RX', 'OTC', 'RX/OTC') ) as t) as n
group by n.Type

In [ ]:
SELECT Type, EaseOfUse,
       COUNT(*) OVER (
            PARTITION BY Type
            ORDER BY EaseOfUse
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) * 1.0 / COUNT(*) OVER (PARTITION BY Type ) AS cumulative_dist,
        cume_dist() over(
            partition by Type
            order by EaseOfUse
        ) as 'percentile_score'
FROM drug
WHERE Type IN ('RX', 'OTC', 'RX/OTC')
ORDER BY Type, cumulative_dist DESC;

In [ ]:
select * from (select t.Condition, percentile_cont(.5) within group (order by Satisfaction) over(partition by t.Condition) as median_sat from drug as t) as n
group by n.Condition
order by n.median_sat desc;

In [ ]:
select t.condition,t.drug,round(price,2) as price,
round(avg(Price)over(partition by t.Condition order by t.drug rows between unbounded preceding and current row ),2) as running_avg_price
from drug as t
ORDER BY t.Condition ASC, t.Drug ASC;

In [ ]:
SELECT drug.Condition, drug.Drug, drug.Reviews,
    (drug.Reviews - LAG(drug.Reviews) OVER (
        PARTITION BY  drug.Drug
        ORDER BY drug.Reviews DESC)
    ) * 100.0 / LAG(drug.Reviews) OVER (
        PARTITION BY  drug.Drug
        ORDER BY drug.Reviews DESC
    ) AS pct_change
FROM drug
ORDER BY pct_change DESC

In [ ]:
with temp as (select t.Type,t.Satisfaction,
round((sum(t.Satisfaction) over (partition by t.Type)) /(sum(t.Satisfaction) over ())*100,2) as percentage
 from drug as t
where Type in ('RX', 'OTC', 'RX/OTC') )
select * from temp
group by Type
order by  percentage asc,Type asc

In [ ]:
select t.Condition,t.Form, t.Drug, t.Effective, sum(t.Effective) OVER(partition by T.Condition, t.Drug order by t.Drug
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS cumulative_sum_effective
from drug as t
order by
    t.Condition ASC,
    t.Form ASC,
    t.Drug ASC

In [ ]:
with temp as (select t.Type, t.EaseOfUse,
avg(t.EaseOfUse) over(partition by t.Type  rows between unbounded preceding and unbounded following) as avgr
from drug as t
where Type in ('RX', 'OTC', 'RX/OTC'))
 select *, dense_rank() over(order by avgr) as rnk   from  temp
group by Type
order by rnk asc

In [ ]:
with temp as (select t.Condition,t.drug,round(t.Reviews,2) as Reviews,round(t.Effective,2)as Effective,
dense_rank()over(partition by t.Condition order by t.Reviews desc )  as rnk,
round(avg(t.Effective) over (partition by t.Condition, t.drug rows between unbounded preceding and unbounded following),2) as avg_effect
from drug as t)
select * from (select * from temp as n
where n.rnk<4) as m